In [1]:
import pandas as pd
from collections import Counter

# Part 1

--- Day 5: Hydrothermal Venture ---

You come across a field of hydrothermal vents on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in lines; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:
```
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
```
Each line of vents is given as a line segment in the format x1,y1 -> x2,y2 where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:

An entry like 1,1 -> 1,3 covers points 1,1, 1,2, and 1,3.
An entry like 9,7 -> 7,7 covers points 9,7, 8,7, and 7,7.
For now, only consider horizontal and vertical lines: lines where either x1 = x2 or y1 = y2.

So, the horizontal and vertical lines from the above list would produce the following diagram:
```
.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
```
In this diagram, the top left corner is 0,0 and the bottom right corner is 9,9. Each position is shown as the number of lines which cover that point or . if no line covers that point. The top-left pair of 1s, for example, comes from 2,2 -> 2,1; the very bottom row is formed by the overlapping lines 0,9 -> 5,9 and 0,9 -> 2,9.

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

Consider only horizontal and vertical lines. At how many points do at least two lines overlap?

In [18]:
df = pd.read_csv('dec5_input.csv')

In [6]:
df.head()

,point1,point2
0,"777,778","777,676"
1,"500,510","378,510"
2,"441,657","441,638"
3,"724,480","724,778"
4,"702,85","44,85"


In [13]:
def preprocess(coords):
    return tuple([int(x) for x in coords.split(",")])

In [19]:
df['point1'] = df.point1.apply(preprocess)
df['point2'] = df.point2.apply(preprocess)

In [66]:
df['points'] = df[['point1', 'point2']].apply(tuple, axis=1)

In [122]:
def generate_points_2(points, consider_diag=False):
    
    # unpack
    point1, point2 = points
    
    x1, y1 = point1
    x2, y2 = point2
    
    # Figure out steps
    x_step = 1
    y_step = 1
          
    if y1 > y2:
        y_step = -1
        
    if x1 > x2:
        x_step = -1
    
    points = []
    
    x_range = range(x1, x2+x_step, x_step)
    y_range = range(y1, y2+y_step, y_step)
    
    # Vertical line
    if x1 == x2:
        x_range = [x1] * len(y_range)
    
    # Horizontal line
    elif y1 == y2:            
        y_range = [y1] * len(x_range)

    # nothing more needs to be done for diag points
    points = points + list(zip(x_range,y_range))
           
    return points

In [124]:
df['all_points'] = df.points.apply(lambda x: generate_points_2(x, False))
df.head()

In [126]:
point_counts = Counter()
df.all_points.apply(lambda x: point_counts.update(x))

0      None
1      None
2      None
3      None
4      None
       ... 
495    None
496    None
497    None
498    None
499    None
Name: points, Length: 500, dtype: object

In [127]:
# ANSWER
len({ x: count for x, count in point_counts.items() if count >= 2 })

6283

# Part 2

--- Part Two ---
Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider diagonal lines.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

An entry like 1,1 -> 3,3 covers points 1,1, 2,2, and 3,3.
An entry like 9,7 -> 7,9 covers points 9,7, 8,8, and 7,9.
Considering all lines from the above example would now produce the following diagram:

```
1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
```

You still need to determine the number of points where at least two lines overlap. In the above example, this is still anywhere in the diagram with a 2 or larger - now a total of 12 points.

Consider all of the lines. At how many points do at least two lines overlap?

In [128]:
df['all_points_w_diag'] = df.points.apply(lambda x: generate_points(x, True))

In [129]:
df.tail()

,point1,point2,points,all_points,all_points_w_diag
495,"(835, 304)","(964, 304)","((835, 304), (964, 304))","[(835, 304), (836, 304), (837, 304), (838, 304...","[(835, 304), (836, 304), (837, 304), (838, 304..."
496,"(480, 249)","(74, 655)","((480, 249), (74, 655))","[(480, 249), (479, 250), (478, 251), (477, 252...","[(480, 249), (479, 250), (478, 251), (477, 252..."
497,"(742, 883)","(213, 883)","((742, 883), (213, 883))","[(742, 883), (741, 883), (740, 883), (739, 883...","[(742, 883), (741, 883), (740, 883), (739, 883..."
498,"(943, 41)","(378, 41)","((943, 41), (378, 41))","[(943, 41), (942, 41), (941, 41), (940, 41), (...","[(943, 41), (942, 41), (941, 41), (940, 41), (..."
499,"(869, 924)","(869, 563)","((869, 924), (869, 563))","[(869, 924), (869, 923), (869, 922), (869, 921...","[(869, 924), (869, 923), (869, 922), (869, 921..."


In [130]:
point_counts_w_diag = Counter()
df.all_points_w_diag.apply(lambda x: point_counts_w_diag.update(x))

0      None
1      None
2      None
3      None
4      None
       ... 
495    None
496    None
497    None
498    None
499    None
Name: all_points_w_diag, Length: 500, dtype: object

In [131]:
# ANSWER
len({ x: count for x, count in point_counts_w_diag.items() if count >= 2 })

18864